# Feature Engineering

In [20]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [21]:
train = pd.read_csv("../data/raw/application_train.csv")
test = pd.read_csv("../data/raw/application_test.csv")

print(train.shape, test.shape)

(307511, 122) (48744, 121)


In [22]:
# AGE feature
train['AGE'] = (-train['DAYS_BIRTH'] / 365).round()
test['AGE']  = (-test['DAYS_BIRTH'] / 365).round()

train['AGE'].describe()

count    307511.000000
mean         43.938646
std          11.964047
min          21.000000
25%          34.000000
50%          43.000000
75%          54.000000
max          69.000000
Name: AGE, dtype: float64

In [23]:
# Sentinel flag
train['EMPLOYED_ANOM'] = (train['DAYS_EMPLOYED'] == 365243).astype(int)
test['EMPLOYED_ANOM']  = (test['DAYS_EMPLOYED'] == 365243).astype(int)

# Replace sentinel with NaN
train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace=True)
test['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace=True)

train[['DAYS_EMPLOYED', 'EMPLOYED_ANOM']].head(10)

,DAYS_EMPLOYED,EMPLOYED_ANOM
0,-637.0,0
1,-1188.0,0
2,-225.0,0
3,-3039.0,0
4,-3038.0,0
5,-1588.0,0
6,-3130.0,0
7,-449.0,0
8,NaN,1
9,-2019.0,0


In [24]:
high_missing_cols = [
    'COMMONAREA_AVG', 'COMMONAREA_MODE', 'COMMONAREA_MEDI',
    'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAPARTMENTS_AVG',
    'FONDKAPREMONT_MODE', 'LIVINGAPARTMENTS_AVG', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAPARTMENTS_MODE',
    'FLOORSMIN_MODE', 'FLOORSMIN_AVG', 'FLOORSMIN_MEDI',
    'YEARS_BUILD_AVG', 'YEARS_BUILD_MODE', 'YEARS_BUILD_MEDI',
    'OWN_CAR_AGE', 'LANDAREA_MEDI', 'LANDAREA_AVG', 'LANDAREA_MODE',
    'BASEMENTAREA_MODE', 'BASEMENTAREA_MEDI', 'BASEMENTAREA_AVG',
    'NONLIVINGAREA_MODE'
]

for col in high_missing_cols:
    train[col + '_MISS'] = train[col].isna().astype(int)
    test[col + '_MISS']  = test[col].isna().astype(int)

train[[c for c in train.columns if c.endswith('_MISS')]].head()

,COMMONAREA_AVG_MISS,COMMONAREA_MODE_MISS,COMMONAREA_MEDI_MISS,NONLIVINGAPARTMENTS_MEDI_MISS,NONLIVINGAPARTMENTS_MODE_MISS,NONLIVINGAPARTMENTS_AVG_MISS,FONDKAPREMONT_MODE_MISS,LIVINGAPARTMENTS_AVG_MISS,LIVINGAPARTMENTS_MEDI_MISS,LIVINGAPARTMENTS_MODE_MISS,...,YEARS_BUILD_MODE_MISS,YEARS_BUILD_MEDI_MISS,OWN_CAR_AGE_MISS,LANDAREA_MEDI_MISS,LANDAREA_AVG_MISS,LANDAREA_MODE_MISS,BASEMENTAREA_MODE_MISS,BASEMENTAREA_MEDI_MISS,BASEMENTAREA_AVG_MISS,NONLIVINGAREA_MODE_MISS
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,1,1,1,...,1,1,0,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [25]:
# Belge sütunlarını listele
flag_doc_cols = [col for col in train.columns if 'FLAG_DOCUMENT' in col]

# Frekansları kontrol et
train[flag_doc_cols].sum().sort_values().head(10)

FLAG_DOCUMENT_12      2
FLAG_DOCUMENT_10      7
FLAG_DOCUMENT_2      13
FLAG_DOCUMENT_4      25
FLAG_DOCUMENT_7      59
FLAG_DOCUMENT_17     82
FLAG_DOCUMENT_21    103
FLAG_DOCUMENT_20    156
FLAG_DOCUMENT_19    183
FLAG_DOCUMENT_15    372
dtype: int64

In [26]:
# Nadir doküman sütunlarını filtrele ve düşür
rare_docs = [col for col in flag_doc_cols if train[col].sum() < 500]

train.drop(columns=rare_docs, inplace=True)
test.drop(columns=rare_docs, inplace=True)

print(f"Dropped {len(rare_docs)} rare document columns.")

Dropped 10 rare document columns.


In [27]:
def rare_category_grouping(df, col, threshold=200):
    value_counts = df[col].value_counts()
    rare_labels = value_counts[value_counts < threshold].index
    df[col] = np.where(df[col].isin(rare_labels), "Other", df[col])
    return df

# Uygulanacak sütunlar
rare_cols = ['ORGANIZATION_TYPE', 'OCCUPATION_TYPE', 'NAME_TYPE_SUITE']

for col in rare_cols:
    train = rare_category_grouping(train, col)
    test  = rare_category_grouping(test, col)

train[rare_cols].nunique()

ORGANIZATION_TYPE    51
OCCUPATION_TYPE      18
NAME_TYPE_SUITE       7
dtype: int64

In [28]:
# Credit to income ratio
train['CREDIT_INCOME_RATIO'] = train['AMT_CREDIT'] / train['AMT_INCOME_TOTAL']
test['CREDIT_INCOME_RATIO']  = test['AMT_CREDIT'] / test['AMT_INCOME_TOTAL']

# Annuity to income ratio
train['ANNUITY_INCOME_RATIO'] = train['AMT_ANNUITY'] / train['AMT_INCOME_TOTAL']
test['ANNUITY_INCOME_RATIO']  = test['AMT_ANNUITY'] / test['AMT_INCOME_TOTAL']

# Annuity to credit ratio (payment_rate)
train['ANNUITY_CREDIT_RATIO'] = train['AMT_ANNUITY'] / train['AMT_CREDIT']
test['ANNUITY_CREDIT_RATIO']  = test['AMT_ANNUITY'] / test['AMT_CREDIT']

# Credit term (indirect number of months)
train['CREDIT_TERM'] = train['AMT_CREDIT'] / train['AMT_ANNUITY']
test['CREDIT_TERM']  = test['AMT_CREDIT'] / test['AMT_ANNUITY']

# Goods price relative to income
train['PRICE_TO_INCOME'] = train['AMT_GOODS_PRICE'] / train['AMT_INCOME_TOTAL']
test['PRICE_TO_INCOME']  = test['AMT_GOODS_PRICE'] / test['AMT_INCOME_TOTAL']

train[['CREDIT_INCOME_RATIO', 'ANNUITY_INCOME_RATIO', 'ANNUITY_CREDIT_RATIO', 'CREDIT_TERM', 'PRICE_TO_INCOME']].head()

,CREDIT_INCOME_RATIO,ANNUITY_INCOME_RATIO,ANNUITY_CREDIT_RATIO,CREDIT_TERM,PRICE_TO_INCOME
0,2.007889,0.121978,0.060749,16.461104,1.733333
1,4.790750,0.132217,0.027598,36.234085,4.183333
2,2.000000,0.100000,0.050000,20.000000,2.000000
3,2.316167,0.219900,0.094941,10.532818,2.200000
4,4.222222,0.179963,0.042623,23.461618,4.222222


In [29]:
age_bins = [20, 28, 35, 50, 60, 75]
age_labels = ['20-28', '28-35', '35-50', '50-60', '60+']

train['AGE_BIN'] = pd.cut(train['AGE'], bins=age_bins, labels=age_labels)
test['AGE_BIN']  = pd.cut(test['AGE'],  bins=age_bins, labels=age_labels)

train[['AGE', 'AGE_BIN']].head(10)

,AGE,AGE_BIN
0,26.0,20-28
1,46.0,35-50
2,52.0,50-60
3,52.0,50-60
4,55.0,50-60
5,46.0,35-50
6,38.0,35-50
7,52.0,50-60
8,55.0,50-60
9,40.0,35-50


In [30]:
# Soft-cap (Winsorization style)
for col in ['CREDIT_INCOME_RATIO', 'ANNUITY_INCOME_RATIO']:
    upper = train[col].quantile(0.999)
    train[col] = np.clip(train[col], None, upper)
    test[col]  = np.clip(test[col], None, upper)

train[['CREDIT_INCOME_RATIO', 'ANNUITY_INCOME_RATIO']].describe()

,CREDIT_INCOME_RATIO,ANNUITY_INCOME_RATIO
count,307511.000000,307499.000000
mean,3.953498,0.180800
std,2.658001,0.093619
min,0.004808,0.000224
25%,2.018667,0.114782
50%,3.265067,0.162833
75%,5.159880,0.229067
max,19.293778,0.727986


In [31]:
# Orijinal high-missing sütunları drop et
train.drop(columns=high_missing_cols, inplace=True)
test.drop(columns=high_missing_cols, inplace=True)

print("Dropped original high-missing columns, kept *_MISS flags.")

Dropped original high-missing columns, kept *_MISS flags.


In [32]:
cols_to_drop = [
    'DAYS_BIRTH',
    'DAYS_EMPLOYED'
]

train.drop(columns=cols_to_drop, inplace=True)
test.drop(columns=cols_to_drop, inplace=True)

print("Dropped:", cols_to_drop)

Dropped: ['DAYS_BIRTH', 'DAYS_EMPLOYED']


In [33]:
# Hedef sütunu ayrı tut
target = 'TARGET'

# Sayısal sütunlar
numeric_cols = train.select_dtypes(include=['int64','float64']).columns.tolist()

# TARGET'i çıkar
numeric_cols = [col for col in numeric_cols if col != target]

# Kategorik sütunlar
categorical_cols = train.select_dtypes(include=['object']).columns.tolist()

print("Numerical:", len(numeric_cols))
print("Categorical:", len(categorical_cols))

Numerical: 76
Categorical: 15


In [34]:
# Sayısal sütunlar için median
for col in numeric_cols:
    train[col].fillna(train[col].median(), inplace=True)
    test[col].fillna(train[col].median(), inplace=True)

# Kategorik sütunlar için mode
for col in categorical_cols:
    train[col].fillna(train[col].mode()[0], inplace=True)
    test[col].fillna(train[col].mode()[0], inplace=True)

# Son kontrol
train[numeric_cols + categorical_cols].isnull().sum().sum()

0

In [35]:
# Train için ohe
train_encoded = pd.get_dummies(train, columns=categorical_cols, drop_first=True)

# Test için ohe
test_encoded = pd.get_dummies(test, columns=categorical_cols, drop_first=True)

print(train_encoded.shape, test_encoded.shape)

(307511, 217) (48744, 189)


In [36]:
# Train ve test kolon setlerini al
train_cols = set(train_encoded.columns)
test_cols = set(test_encoded.columns)

# Sadece train'de olup test'te olmayan kolonlar
missing_in_test = train_cols - test_cols

# Sadece test'te olup train'de olmayan kolonlar
missing_in_train = test_cols - train_cols

# Test'e eksik kolonları ekle
for col in missing_in_test:
    test_encoded[col] = 0

# Train'de olmayan kolonları test'ten drop et
test_encoded.drop(columns=list(missing_in_train), errors='ignore', inplace=True)

# Son olarak sütunları aynı sıraya sok
test_encoded = test_encoded[train_encoded.columns]

print("After alignment:")
print(train_encoded.shape, test_encoded.shape)

After alignment:
(307511, 217) (48744, 217)


In [37]:
train_encoded.to_csv("../data/processed/train_fe.csv", index=False)
test_encoded.to_csv("../data/processed/test_fe.csv", index=False)

print("Processed datasets saved.")

Processed datasets saved.


## Çıkarım #11 — Feature Engineering Sonrası Veri Saklama

- Feature engineering aşamasından sonra veri setinin işlenmiş hali data/processed/ dizinine kaydedildi.

- Bu yaklaşım, modelleme aşamasının deterministik ve tekrar üretilebilir olmasını sağlar.

- Eğitim ve test veri setleri aynı kolon yapısına sahiptir; model eğitiminde kolon uyuşmazlığı hatası oluşmaz.

- İşlenmiş verinin saklanması, modeling defterinin feature engineering’e bağımlı olmamasını sağlayarak akademik çalışma yapısını temizlemiştir.

- TARGET sütunu yalnızca eğitim setinde tutulmuştur; bu supervised eğitim protokolüne uygundur.

# Genel Yorum

- Ham formatta anlamsız olan DAYS_BIRTH sütunu kaldırıldı ve yerine işaret yönünü normalize eden AGE değişkeni kullanıldı.

- DAYS_EMPLOYED sütunundaki sentinel (365243) değeri tanımlandı ve bunun kredi geri ödemesi yapmayan segmentle farklı bir ilişki taşıdığı gözlendi. Bu davranış EMPLOYED_ANOM isimli binary feature ile korundu.

- Yüksek eksik oranlı demografik ve konut özellikleri ham değer olarak düşürüldü; ancak veri yokluğunun davranışsal sinyali _MISS flag sütunlarıyla korundu.

- Kredi–gelir ilişkisi, kredi–mal bedeli oranı, kredi süresi gibi domain odaklı türev feature’lar (CREDIT_INCOME_RATIO, ANNUITY_INCOME_RATIO, PRICE_TO_INCOME, CREDIT_TERM) modele eklenerek daha güçlü ayrım kapasitesi sağlandı.

- Belgelerle ilgili nadir görülen sütunlar düşürüldü, böylece model gürültüden kaçındı.

- AGE_BIN kategorik segmentasyonu eklendi; daha yaşlı bireylerde risk azalması gözlendi.

- Tüm kategorik özellikler One-Hot Encoding ile modele uygun hale getirildi.

- Train–test kolonları hizalandı; bu adım modellerin tahmin sırasında hata vermesini engeller.

- Tüm NaN değerler median/mode imputasyonu ile dolduruldu; bu stabilite artışı ve eğitim sürecinde hata riskinin ortadan kalkması anlamına gelir.